In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -r /content/drive/MyDrive/TG/tg2_COVIDNet/requirements.txt

     |████████████████████████████████| 133kB 4.2MB/s 
     |████████████████████████████████| 174kB 34.2MB/s 
     |████████████████████████████████| 153kB 49.6MB/s 
     |████████████████████████████████| 163kB 45.3MB/s 
     |████████████████████████████████| 829kB 45.6MB/s 
     |████████████████████████████████| 122kB 45.6MB/s 
     |████████████████████████████████| 4.0MB 35.1MB/s 
     |████████████████████████████████| 122kB 40.4MB/s 
     |████████████████████████████████| 788kB 44.0MB/s 
     |████████████████████████████████| 122kB 40.3MB/s 
     |████████████████████████████████| 1.4MB 45.1MB/s 
     |████████████████████████████████| 133kB 37.3MB/s 
     |████████████████████████████████| 307kB 45.5MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 112kB 41.3MB/s 
     |████████████████████████████████| 92kB 11.2MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 


In [1]:
train_ds_path = "../target_dataset/train"
test_ds_path = "../target_dataset/test"


In [2]:
batch_size = 32
image_size = (240, 240)

from tensorflow.keras.preprocessing import image_dataset_from_directory

test_ds = image_dataset_from_directory(
    test_ds_path,
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=image_size,
    shuffle=False,
)

#Junta todos os batches
test_ds = test_ds.unbatch()


# TF tensors -> numpy array
test_ds = test_ds.as_numpy_iterator()

In [3]:
encoding = {"COVID-19": 2, "Normal": 0, "Pneumonia": 1}

In [ ]:
import cv2
import os
import numpy as np

img_array_train = []
target_array_train = []

for c in encoding:
    files = list(f for f in os.listdir(os.path.join(train_ds_path, c)) if os.path.isfile(os.path.join(train_ds_path, c, f)))
    for f in files:
        img = cv2.imread(os.path.join(train_ds_path, c, f), cv2.IMREAD_GRAYSCALE)
        # img = np.expand_dims(img, axis=2)
        img = cv2.resize(img, image_size)
        img_array_train.append(img)
        target_array_train.append(encoding[c])
    files = list(f for f in os.listdir(os.path.join(test_ds_path, c)) if os.path.isfile(os.path.join(test_ds_path, c, f)))
    for f in files:
        img = cv2.imread(os.path.join(test_ds_path, c, f), cv2.IMREAD_GRAYSCALE)
        # img = np.expand_dims(img, axis=2)
        img = cv2.resize(img, image_size)
        img_array_test.append(img)
        target_array_test.append(encoding[c])


In [ ]:
from random import shuffle

data_train = list(zip(img_array_train, target_array_train))
shuffle(data_train)
X_train, y_train = zip(*data_train)

data_test = list(zip(img_array_test, target_array_test))
shuffle(data_test)
X_test, y_test = zip(*data_test)

In [ ]:
import numpy as np

X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
import os
from numpy import save

ds_arrays_path = '/content/drive/MyDrive/TG/dataset_arraysTRAINING'
# ds_arrays_path = '/home/beduinigor/dataset_arraysTRAINING'

save(os.path.join(ds_arrays_path, 'x_train_data.npy'), X_train)
save(os.path.join(ds_arrays_path, 'y_train_data.npy'), y_train)
save(os.path.join(ds_arrays_path, 'x_test_data.npy'), X_test)
save(os.path.join(ds_arrays_path, 'y_test_data.npy'), y_test)

In [4]:
import os
from numpy import load

ds_arrays_path = '/content/drive/MyDrive/TG/dataset_arraysTRAINING'
# ds_arrays_path = '/home/beduinigor/dataset_arraysTRAINING'

X_train = load(os.path.join(ds_arrays_path,'x_train_data.npy'))
y_train = load(os.path.join(ds_arrays_path,'y_train_data.npy'))
X_test = load(os.path.join(ds_arrays_path,'x_test_data.npy'))
y_test = load(os.path.join(ds_arrays_path,'y_test_data.npy'))

In [5]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)


In [6]:
import tensorflow as tf

tensorboard_training_path = "/content/drive/MyDrive/TG/training/tensorboard/kfold"

early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor="loss",
    min_delta=0.005,
    patience=7,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True)

tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir=tensorboard_training_path,
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    update_freq="epoch",
    profile_batch=2,
    embeddings_freq=0,
    embeddings_metadata=None)

callbacks = [early_stop_callback, tb_callback]

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import autokeras as ak
import numpy as np
import os

kfold_training_path = "/content/drive/MyDrive/TG/training/models/kfold"
model_path = "/content/drive/MyDrive/TG/tg2_COVIDNet/models/final_model.h5"

for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    n_fold = i + 1
    print(f"\n\n\nFOLD {n_fold}")
    print("-------------------------------")

    model = load_model(model_path, custom_objects=ak.CUSTOM_OBJECTS)
    config = model.get_config()
    clean_model = Model.from_config(config)
    clean_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    fold_path = os.path.join(kfold_training_path, f"fold_{n_fold}")
    if not os.path.isdir(fold_path):
        os.mkdir(fold_path)
    if not os.path.isdir(tensorboard_training_path):
        os.mkdir(tensorboard_training_path)


    x_train_fold = X_train[train_index]
    y_train_fold = to_categorical(y_train[train_index], 3)

    x_test_fold = X_train[test_index]
    y_test_fold = to_categorical(y_train[test_index], 3)

    print("Saving training fold nparrays...")
    np.save(os.path.join(fold_path, f"x_train_arr_fold_{n_fold}.npy"), x_train_fold)
    np.save(os.path.join(fold_path, f"y_train_arr_fold_{n_fold}.npy"), y_train_fold)
    np.save(os.path.join(fold_path, f"x_test_arr_fold_{n_fold}.npy"), x_test_fold)
    np.save(os.path.join(fold_path, f"y_test_arr_fold_{n_fold}.npy"), y_test_fold)
    print("Arrays saved with sucess!")

    print("\nStart training...")  
    print("Class weights:")
    class_weight = compute_class_weight(class_weight='balanced', classes=np.unique(y_train[train_index]), y=y_train[train_index])
    class_weight = {i: class_weight[i] for i in range(len(class_weight))}
    print(class_weight)

    clean_model.fit(x=x_train_fold,
              y=y_train_fold,
              batch_size=batch_size,
              epochs=100,
              validation_data=(x_test_fold, y_test_fold),
              callbacks=callbacks,
              class_weight=class_weight)
    
    print("\nSaving model...")
    try:
        clean_model.save(os.path.join(fold_path, f"model_fold_{n_fold}.h5"), save_format='h5')
        print("Model saved with sucess!")
    except:
        print("Fail trying to save the model!")




FOLD 1
-------------------------------
Saving training fold nparrays...
Arrays saved with sucess!

Start training...
Class weights:
{0: 0.5858015984955336, 1: 0.8537170263788969, 2: 8.224422442244224}
Epoch 1/100
  1/312 [..............................] - ETA: 0s - loss: 1.8314 - accuracy: 0.3750WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
312/312 [==============================] - 305s 976ms/step - loss: 0.9592 - accuracy: 0.5919 - val_loss: 0.8363 - val_accuracy: 0.6481
Epoch 2/100
312/312 [==============================] - 300s 960ms/step - loss: 0.7621 - accuracy: 0.7035 - val_loss: 6.3029 - val_accuracy: 0.0445
Epoch 3/100
312/312 [==============================] - 300s 961ms/step - loss: 0.7118 - accuracy: 0.7246 - val_loss: 5.5571 - val_accuracy: 

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

from tensorflow.keras.models import load_model
import numpy as np
import os

ds_arrays_path = '/content/drive/MyDrive/TG/dataset_arraysTRAINING'

for root, subdirs, files in os.walk(kfold_training_path):
    for dir in subdirs:
        fold_path = os.path.join(root, dir)
        for f in os.listdir(fold_path):
            model_path = os.path.join(fold_path, f)
            if "model" in f:
                model = load_model(model_path)

                X_test = load(os.path.join(ds_arrays_path,'x_test_data.npy'))
                y_test = load(os.path.join(ds_arrays_path,'y_test_data.npy'))

                predictions = model.predict(x=X_test)
                num_predictions = np.argmax(predictions, axis=-1)
                print(classification_report(y_test, num_predictions))
                print(confusion_matrix(y_test, num_predictions))
                # print(average_precision_score(y_test, num_predictions))
        # break